В данном ноутбуке рассмотрены несколько современных моделей, основанных на базовой архитектуре "Трансформер". BERT (или Bidirectional Encoder Representations from Transformers) - модель, которая позволяет дофайнтьюнить заранее обученные при помощи MLM эмбеддинги. На данный момент существует много вариаций данной модели, включая многоязычные.   Здесь будут рассмотрены несколько современнных архитектур, применяемых для решения сложных задач в сфере нлп.

Прежде чем переходить к основной части задачи необходимо, собственно, более подробно описать ее. Jigsaw Multilingual Toxic Comment Classification - это задача по классификации комментариев на два вида: токсичные и нетоксичные. Фактически, эта задача представляет собой бинарную классификацию на 2 класса: токсичные (1) и нетоксичные (0). Таким образом, мы предсказываем вероятность того, что комментарий окажется токсичным. Особенностью данного соревнования является то, что в тренировочной выборке присутствуют только англоязычные комментарии, а вот в валидационном и треничовочном датасетах есть комментарии на разных языках. Теперь приступим к обучению самих моделей.

Из-за особенностей kaggle-only соревнований, с помощью одного ноутбука можно получить только один сабмит, поэтому здесь представлены только результаты обучения разных моделей: финальный сабмит был получен с помощью другого ноутбука, который также приложен к этому дз.  Непосредственно на кэггл данный ноутбук запусался 3 раза, чтобы сохранить предсказание каждой модели отдельно (код остальных был закомментирован) и засабмитить его

# Модель 1

## xlm roberta large aka самая лучшая модель

Было изучено много ноутбуков - самую высокую точность демонстрировала именно данная модель или же ансамбль на ее базе - она является самой современной (СОТА) для решения подобных задач. Мой лучший сабмит также был получен с помощью данной модели. Одна обученная модель xlm roberta large c правильно подобранными параметрами позволяет уже достичь качества приблизительно 0.936. Мой самый лучший сабмит был получен с помощью взятия среднего 4 моделей данного типа с разными параметрами и 2 разными видами обработки (базовая и более тщательная с удалением всех ненужных символов, которые по идее превносят шум).

Тут представлен, собственно, достаточно базовый пайплайн обработки данных и обучения модели

In [0]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

Функция для предобработки данных и приведения их к формату, который использует BERT. Здесь текст каждого комментария кодируется специальным токенайзером, который возвращает уже числовые значения, также для токенайзера устанавливается определенная максимальная длина и, чтобы запихнуть разные по длине предложения в батчи одного размера, добавляется паддинг.

In [0]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

А эта функция создает саму модель. Первый слой - инпут с шейп, равной максимальной длине последовательности, затем слой с самой моделью, возвращающий hidden_state трансформера, что и будет являться нашим эмбеддингом, ну а затем прогоняем все через полносвязный слой с выходной размерностью, равной количеству классов (2 в данном случае), который уже и выдает нужную вероятность того, что комментарий toxic.

In [0]:
def build_model(transformer, max_len=512, lr=1e-5, loss='binary_crossentropy'):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=lr), loss=loss, metrics=['accuracy', tf.keras.metrics.AUC()])
    
    
    return model

Настраиваем TPU

In [0]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


Блок гиперпараметров, которые можно варьировать для тюнинга

In [0]:
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'


Теперь, когда функции определены, можно приступать непосредственно к подготовке данных и обучению.

In [0]:
# сам токенайзер
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Загружаем данные из кэггл

In [0]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

# добавляем подмножество из втрого тренировочного датасета в 1
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

Применяем функцию предобработки

In [0]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

CPU times: user 7min 28s, sys: 2.01 s, total: 7min 30s
Wall time: 7min 30s


Оборачиваем обычные эррэи в тензорфлоу датасет, чтобы удобно работать с батчами

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

Создаем саму модель

In [0]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 2min 3s, sys: 38.9 s, total: 2min 42s
Wall time: 2min 41s


Дальше идет обучение. во всех изученных мной ноутбуках схема была примерно такая: 2-4 эпохи обучения на обучающей выборке, еще 2-4 на валидационной. Здесь также - сначала обучение на трейн:

In [0]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 3404 steps, validate for 63 steps
Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


3404/3404 [==============================] - 1839s 540ms/step - loss: 0.0696 - accuracy: 0.9722 - auc: 0.9956 - val_loss: 0.4936 - val_accuracy: 0.8463 - val_auc: 0.4896
Epoch 2/2
3404/3404 [==============================] - 1631s 479ms/step - loss: 0.0664 - accuracy: 0.9752 - auc: 0.9959 - val_loss: 0.2827 - val_accuracy: 0.8714 - val_auc: 0.9062


А теперь еще несколько эпох на валидационной части

In [0]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

Train for 62 steps
Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


62/62 [==============================] - 122s 2s/step - loss: 0.2476 - accuracy: 0.8881 - auc: 0.9191
Epoch 2/2
62/62 [==============================] - 140s 2s/step - loss: 0.1807 - accuracy: 0.9196 - auc: 0.9583


Сабмит одной модели xlm roberta large готов

In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission.csv', index=False) 

А вот и результат, который достаточно высок 
![alt text](https://sun9-51.userapi.com/c813024/v813024533/d78f8/q94JZBD5HEk.jpg)

## модель 2 distilbert 

Пайплайн здесь точно такой же, мы просто заменяем "сердце" нашей модели (трансформер) на distilbert. Обрабатываем данные нужным токенайзером, оборачиваем в датасет, создаем новую модель с такой же архитектурой, но с distilbert внутри. К сожалению, нет варианта bert (или distilbert) c hidden layer 1024: модель слабее (768). Обучаем на трейне и валидации - в данном случае 4 эпохи. 

In [0]:
MODEL = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

создаем датасет

In [0]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

CPU times: user 19min 31s, sys: 1.01 s, total: 19min 32s
Wall time: 19min 31s


In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

Создаем модель и обучаем

In [0]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 768)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 33.7 s, sys: 10.1 s, total: 43.8 s
Wall time: 47.1 s


In [0]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS*2
)

Train for 3404 steps, validate for 63 steps
Epoch 1/4
3404/3404 [==============================] - 432s 127ms/step - loss: 0.0795 - accuracy: 0.9696 - auc_1: 0.9941 - val_loss: 0.3943 - val_accuracy: 0.8446 - val_auc_1: 0.7907
Epoch 2/4
3404/3404 [==============================] - 375s 110ms/step - loss: 0.0590 - accuracy: 0.9768 - auc_1: 0.9968 - val_loss: 0.4962 - val_accuracy: 0.8469 - val_auc_1: 0.8190
Epoch 3/4
3404/3404 [==============================] - 375s 110ms/step - loss: 0.0510 - accuracy: 0.9797 - auc_1: 0.9976 - val_loss: 0.6480 - val_accuracy: 0.8469 - val_auc_1: 0.7588
Epoch 4/4
1982/3404 [================>.............] - ETA: 2:36 - loss: 0.0655 - accuracy: 0.9731 - auc_1: 0.9953

In [0]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Train for 62 steps
Epoch 1/4
62/62 [==============================] - 30s 487ms/step - loss: 0.3754 - accuracy: 0.8506 - auc_1: 0.7757
Epoch 2/4
62/62 [==============================] - 24s 393ms/step - loss: 0.2429 - accuracy: 0.8923 - auc_1: 0.9187
Epoch 3/4
62/62 [==============================] - 7s 108ms/step - loss: 0.1788 - accuracy: 0.9201 - auc_1: 0.9592
Epoch 4/4
62/62 [==============================] - 7s 112ms/step - loss: 0.1152 - accuracy: 0.9508 - auc_1: 0.9839


In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission.csv', index=False)

Несмотря на высокие показатели при мониторинге на трейн и тест, на паблик качество хуже:

![alt text](https://sun9-53.userapi.com/c813024/v813024533/d791c/EVQz1sZe_tM.jpg)

## модель 3 xlm 


К сожалению, погуглив multilingual models, я поняла, что их выбор не особо широк: hugging face предложили, собственно, Берт ( distilbert - его вариация соответственно), XLM-Роберта и XLM, но последнюю модель я не встречала ни в одном из кернелов, так что результаты непредсказуемы



In [0]:
MODEL = 'xlm-mlm-100-1280'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [0]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values


CPU times: user 21min 16s, sys: 3.48 s, total: 21min 19s
Wall time: 21min 19s


In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

Создание модели и  процесс обучения. Так как нельзя запустить все 3 модели в одном кернеле, чтобы сразу сохранить все аутпуты (Resources Exhausted error) - эти части закомментированы, прикладываю скрины 


In [0]:
# %%time
# with strategy.scope():
#     transformer_layer = TFAutoModel.from_pretrained(MODEL)
#     model = build_model(transformer_layer, max_len=MAX_LEN)
# model.summary()

In [0]:
# n_steps = x_train.shape[0] // BATCH_SIZE
# train_history = model.fit(
#     train_dataset,
#     steps_per_epoch=n_steps,
    
#     validation_data=valid_dataset,
#     epochs=EPOCHS
# )


In [0]:
# n_steps = x_valid.shape[0] // BATCH_SIZE
# train_history_2 = model.fit(
#     valid_dataset.repeat(),
#     steps_per_epoch=n_steps,
#     epochs=EPOCHS
# )

In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission.csv', index=False)

Архитектура модели:

![alt text](https://sun9-4.userapi.com/c857528/v857528479/205d53/BBgTF3jllYw.jpg)

Процесс обучения на трейн:

![alt text](https://sun9-41.userapi.com/c857528/v857528479/205d65/YOqtQz7ys0w.jpg)

Процесс обучения на валид:

![alt text](https://sun9-68.userapi.com/c857528/v857528479/205d5c/God1-ks_caU.jpg)



Качество на паблик в итоге лучше, чем у distilbert, но сильно уступает Robertta:
![alt text](https://sun9-28.userapi.com/c857528/v857528479/205d6e/GxTfDAYXcG4.jpg)

# Краткие выводы, касательно данной задачи и моделей, примененных для ее решения

Главной особенностью задачи является то, что в множестве, на котором происходит обучение, состоит из комментариев на английском языке, а вот валидационный и тестовый - включает в себя много разных языков:
Это существенно затрудняет задачу, так как модель, обученная на одном домене (языке), по сути, должна решать задачу для других доменов. Это значит, что, несмотря на разнообразие современны моделей трансформеров, их выбор снижается лишь до малтилингуал (всего 3 модели, указаны в списке hugging face), так как остальные обучаются на конкретном языке и хорошо работают лишь на нем. Берт в данном случае является хорошей базовой моделью, но, тем не менее, он не совершенен: cross lingual models (xlm) - следующий шаг для решения данной задачи, в который добавлены некоторые улучшения, помогающие достичь лучшего качества. В данной задаче distilbert  оказался хорошим бейзлайном: он показал неплохое, но далеко не эталонное качество, xlm оказался лучше, ну а самый лучший же результат продемонстрировала Роберта.
 

In [0]:
res = pd.DataFrame(list(zip(['xlm-roberta-large', 'xlm-mlm-100-1280', 'distilbert-base-multilingual-cased'], [0.9362, 0.8884, 0.8718])), columns=['model', 'test_auc_public'])

In [0]:
res

,model,test_auc_public
0,xlm-roberta-large,0.9362
1,xlm-mlm-100-1280,0.8884
2,distilbert-base-multilingual-cased,0.8718


 References:

Все открытые ноутбуки на кэггл по соревнованию. Большинство из них очень сильно похожи и повторяют тот же самый пайплайн, примененный здесь. Было просмотрено большое количество кернелов и основа была взята из них.